# Проектная работа по предсказанию победителя в Dota2

## Подход 1: градиентный бустинг

### 1. Считываем таблицу с признаками и удаляем признаки, связанные с итогами матча.

In [2]:
import numpy as np
import pandas
features = pandas.read_csv('features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


In [6]:
features.drop(columns=['tower_status_radiant','tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], inplace=True)

### 2. Проверяем наличие пропусков

In [7]:
counts = features.count()
counts.loc[counts != features.shape[0]]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Событие "первая кровь" не во всех играх успело произойти за первые 5 минут, поэтому признаки first_blood_time и first_blood_team имеют пропущенные значения.

### 3. Заменим пропуски на нули

In [8]:
features.fillna(0, inplace=True)

In [9]:
counts = features.count()
counts.loc[counts != features.shape[0]]

Series([], dtype: int64)

### 4. Определим целевую переменную и признаки

In [10]:
y = features.loc[:, 'radiant_win']

In [11]:
X = features.loc[:, 'start_time':'dire_first_ward_time']

### 5. Обучим градиентный бустинг над деревьями

In [12]:
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(n_splits=5, shuffle=True, random_state=241)

In [25]:
from sklearn.ensemble import GradientBoostingClassifier

In [26]:
import time
import datetime

for n_estimator in [10, 20, 30, 40, 100, 250]
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators = n_estimator, max_depth = 2, random_state=241)
    scores = cross_val_score(clf, X, y, scoring='roc_auc', cv=cv)
    print('n_estimator=', n_estimator, ' Time elapsed:', datetime.datetime.now() - start_time)
    scores.mean()

Time elapsed: 0:14:22.377059


### Отчет к подходу "градиентный бустинг":
1. Признаки имеющие пропуски в значениях:
    - first_blood_time
    - first_blood_team               
    - first_blood_player1
    - first_blood_player2
    - radiant_bottle_time
    - radiant_courier_time
    - radiant_flying_courier_time
    - radiant_first_ward_time
    - dire_bottle_time
    - dire_courier_time
    - dire_flying_courier_time
    - dire_first_ward_time
    
    Событие "первая кровь" не во всех играх успело произойти за первые 5 минут, поэтому признаки first_blood_time и first_blood_team имеют пропущенные значения.
2. Целевая переменная содержится в столбце "radiant_win"
3. Кросс-валидация для градиентного бустинга с 30 деревьями проводилась 200 секунд. Качество по метрике roc_auc получилось 0.69
4. Имеет смысл использовать более 30 деревьев в градиентном бустинге, т.к при их увеличении качество улучшается и стремится к значению 0.72. Чтобы ускорить его обучение при увеличении количества деревьев можно использовать для обучения и кросс-валидации не всю выборку, а некоторое ее подмножество.

## Подход 2: логистическая регрессия

### 1. Обучим классификатор "логическая регрессия" и найдем наилучший параметр регуляризации С

In [14]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
for C in [1, 10, 100]:
    clf = LogisticRegressionCV(Cs=C, scoring='roc_auc', cv=cv, random_state=241).fit(X,y)
    print("C={:.2f} roc_auc={:.2f}".format(C, clf.score(X, y)))

C=1.00 roc_auc=0.51
C=10.00 roc_auc=0.71
C=100.00 roc_auc=0.71


**Без нормалзации признаков:**

Лучший параметр регуляризации С=10, что соответствует качеству 0.71.
Логистическая регрессия работает значительно быстрее градиентного бустинга, а ее качество немного лучше.

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [16]:
for C in [1, 10, 100]:
    clf = LogisticRegressionCV(Cs=C, scoring='roc_auc', cv=cv, random_state=241).fit(X_scaled,y)
    print("C={:.2f} roc_auc={:.2f}".format(C, clf.score(X_scaled, y)))

C=1.00 roc_auc=0.71
C=10.00 roc_auc=0.72
C=100.00 roc_auc=0.72


**С нормалзацией признаков:**

Лучший параметр регуляризации С=10, что соответствует качеству 0.72.

### 2. Уберем категориальные признаки из выборки, и проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации.

In [17]:
X.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], inplace=True)

In [18]:
for C in [1, 10, 100]:
    clf = LogisticRegressionCV(Cs=C, scoring='roc_auc', cv=cv, random_state=241).fit(X,y)
    print("C={:.2f} roc_auc={:.2f}".format(C, clf.score(X, y)))

C=1.00 roc_auc=0.51
C=10.00 roc_auc=0.71
C=100.00 roc_auc=0.71


Качество практически не изменилось, потому что категориальные признаки нельзя сравнить и поэтому в данном виде они не учитываются при построении можели.

### 3. Выясним сколько различных идентификаторов героев существует в данной игре.

In [19]:
heroes = set(
    list(features['r1_hero'].unique()) + 
    list(features['r2_hero'].unique()) + 
    list(features['r3_hero'].unique()) + 
    list(features['r4_hero'].unique()) + 
    list(features['r5_hero'].unique()) + 
    list(features['d1_hero'].unique()) + 
    list(features['d2_hero'].unique()) + 
    list(features['d3_hero'].unique()) + 
    list(features['d4_hero'].unique()) + 
    list(features['d5_hero'].unique())
)
max(heroes)

112

### 4. Закодируем информацию о героях с "помощью мешка" слов

In [20]:
X_pick = np.zeros((features.shape[0], 112))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1


In [21]:
for i in range(112):
    col_name = 'hero_%i' % i
    X[col_name] = X_pick[:, i]

### 5. Проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации

In [22]:
scaler = StandardScaler()
X_scaled_with_cat = scaler.fit_transform(X)

In [23]:
for C in [1, 10, 100]:
    clf = LogisticRegressionCV(Cs=C, scoring='roc_auc', cv=cv, random_state=241).fit(X_scaled_with_cat,y)
    print("C={:.2f} roc_auc={:.2f}".format(C, clf.score(X_scaled_with_cat, y)))

C=1.00 roc_auc=0.75
C=10.00 roc_auc=0.75
C=100.00 roc_auc=0.75


Качество улучшилось с 0.72 до 0.75. Это связано с тем, что мы стали учитывать такой важный параметр как выбор героя.

### 6. Построим предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled_with_cat, y, test_size=0.8, random_state=241)
clf = LogisticRegression(C=10, random_state=241).fit(X_train,y_train)

In [27]:
clf.predict_proba(X_test)[:, 1]

array([0.33868016, 0.80847888, 0.70817257, ..., 0.87148345, 0.3259469 ,
       0.82760444])

In [28]:
list(clf.predict_proba(X_test)[:, 1])

[0.3386801642621528,
 0.8084788806287313,
 0.7081725676152962,
 0.17208964186152145,
 0.1155835503544421,
 0.08739830051582896,
 0.22625161848942008,
 0.3808382749275668,
 0.6186978760962851,
 0.6257642665133265,
 0.4545304241150112,
 0.4304527205863471,
 0.7399138748598081,
 0.45124277028529636,
 0.7573157640313234,
 0.48777091744255985,
 0.18624186721350366,
 0.22320654094677927,
 0.7875587038943993,
 0.8649699821857304,
 0.4556504859594393,
 0.699187985686977,
 0.5754552970393546,
 0.5650922414160369,
 0.5623321083069246,
 0.6123415781805507,
 0.9547789133968375,
 0.5961409934460143,
 0.2764932713938972,
 0.5080698768473862,
 0.39626895546865437,
 0.7145160338486224,
 0.39242981290054535,
 0.4139795517512192,
 0.31644514199348805,
 0.33932322858761976,
 0.14059059993797726,
 0.5620290868796437,
 0.5167121440291051,
 0.9047524216214677,
 0.15079383207628327,
 0.45561952517418125,
 0.3561853209512365,
 0.29519087238856223,
 0.7568266861711973,
 0.6459816958351529,
 0.4847403807405873,

In [36]:
clf.score(X_test,y_test)

0.6789956803455723

In [37]:
max(list(clf.predict_proba(X_test)[:, 1]))

0.9983627240639319

In [38]:
min(list(clf.predict_proba(X_test)[:, 1]))

0.000853910739723726

### Отчет к подходу "логистическая регрессия":
1. Лучшее качество достигается с нормалзацией признаков и параметром регуляризации С=10, что соответствует качеству 0.72.
    
    Логистическая регрессия работает значительно быстрее градиентного бустинга, а ее качество немного лучше.
2. Удаление категориальных признаков практически не влияет на качество логистической регрессии(0.71 против 0.72), потому что категориальные признаки нельзя напрямую сравнивать и поэтому в данном виде они не учитываются при построении можели.

3. Всего в игре существет 112 различных идентификаторов героев.

4. После добавления "мешка слов" по героям качество модели улучшилось с 0.72 до 0.75. Это связано с тем, что все герои разные и некоторые выигрывают игры чаще других. Поэтому информация о выборе героев улучшает нашу модель.

5. Максимальное значение прогноза на тестовой выборке равно 0.998, а минимальное 0.001

## Проверка финальной модели

In [9]:
features_test = pandas.read_csv('features_test.csv', index_col='match_id')
features_test.fillna(0, inplace=True)

In [8]:
X_pick = np.zeros((features_test.shape[0], 112))
for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick[i, features_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
for i in range(112):
    col_name = 'hero_%i' % i
    features_test[col_name] = X_pick[:, i]


AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [7]:
print(features_test)

[[ 1.43028792e+09  0.00000000e+00  9.30000000e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.43029336e+09  1.00000000e+00  2.00000000e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.43030177e+09  1.00000000e+00  1.12000000e+02 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]
 ...
 [ 1.45022359e+09  1.00000000e+00  8.50000000e+01 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.45024477e+09  0.00000000e+00  7.00000000e+00 ...  0.00000000e+00
   0.00000000e+00 -1.00000000e+00]
 [ 1.45025543e+09  1.00000000e+00  8.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]


In [6]:
features_test.drop(columns=['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], inplace=True)

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [93]:
features_test.shape

(17177, 203)

In [92]:
features_test_scaled = scaler.transform(features_test)

In [94]:
radiant_win = clf.predict_proba(features_test_scaled)[:, 1]

df_results = pandas.DataFrame(data = {'match_id': features_test.index, 'radiant_win': radiant_win})
df_results.head()

,match_id,radiant_win
0,6,0.823291
1,7,0.767529
2,10,0.210232
3,13,0.890992
4,16,0.272952


In [96]:
df_results.to_csv('results.csv', index=False)